The purpose of this notebook is to see the placement of scooters during Nashville rush hours (7am - 9am and 4pm - 6pm) and their relative location to bus stops in the Promise Zone. With this notebook, I aim to answer the following questions: 
    1. Where is there an accumulation of scooters during these time periods and where may scooters be lacking? 
    2. Where are the optimal areas to keep/move scooters in order to aid with the "last mile"?
    3. What is the optimal number of scooters to have in these areas? 

First, we will import the needed packages. 

In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMapWithTime
import datetime as dt

Import and inspect the promise zone json file and the May, June and July datasets. These datasets are subsets of the stationary scooter files that include data from only the above-mentioned timeframes. I'll start with May and then repeat the steps for the other datasets. 

In [ ]:
promise_zone = gpd.read_file('../data/MDHA_Promise_Zones/promise_zone.json')
print(promise_zone.crs)
promise_zone

In [ ]:
may_rush_hour = pd.read_pickle('../data/may_rush_hour.pkl')
may_rush_hour.head()

Create a geometry column for may_rush_hour to be able to create a geodataframe to use with our map.

In [ ]:
may_rush_hour['geometry'] = may_rush_hour.apply(lambda x: Point((x.longitude, 
                                                                x.latitude)), 
                                                                    axis=1)
may_rush_hour.head(3)

Now, I'll create a geodataframe for the May data that matched the CRS on of the Promise Zone dataframe.

In [ ]:
may_geo = gpd.GeoDataFrame(may_rush_hour, 
                           crs = promise_zone.crs, 
                           geometry = may_rush_hour['geometry'])
may_geo.head()

I want to subset the may_geo dataframe into 2 new dataframes containing the morning scooters and the evening scooters. The reasoning behind this is because theoretically we would want scooters available near neighborhoods in the morning to facilitate rides to bus stops, and near bus stops in the evenings to facilitate rides home. 

In [ ]:
may_geo_morning = may_geo[may_geo.hour <= 9]
may_geo_morning.head()

In [ ]:
may_geo_evening = may_geo[may_geo.hour >= 16]
may_geo_evening.head()

Now that the May data frame is done, I will read in the bus stops data, create a geodataframe for it, and use a spatial join to merge it with the Promise Zone dataframe, as we only want to show bus stops in this area. 

In [ ]:
bus_stops = pd.read_csv('../data/busstops_cleaned.csv')
bus_stops.head()

Create the geometry for the bus stops dataframe.

In [ ]:
bus_stops['geometry'] = bus_stops.apply(lambda x: Point((x.lng, 
                                                         x.lat)), 
                                        axis=1)
bus_stops.head(3)

Create a geodatafram of the bus stops data and match the CRS to the Promise Zone geodataframe.

In [ ]:
bus_geo = gpd.GeoDataFrame(bus_stops, 
                           crs = promise_zone.crs, 
                           geometry = bus_stops['geometry'])
bus_geo.head()

Perform a spatial join on the bus_geo and the promise_zone dataframes.

In [ ]:
stops_in_promise = gpd.sjoin(bus_geo, promise_zone, op = 'within')
stops_in_promise.head()

Now that the dataframes are done, I will start to plot the heatmap. First, I need to find a focal point for the map to start. Because the Promise Zone has 6 separate zones, I'll try to find the centermost point of the centermost zone.

In [ ]:
promise_zone.plot();

In [ ]:
promise_zone.geometry[0]

In [ ]:
center = promise_zone.geometry.centroid[0]
area_center = [center.y, center.x]

Using the area_center as the center point of the map, I'll use it to build the maps. First is the map showing the locations of scooters in the morning over time. 

In [ ]:
import folium.plugins as plugins

# Create time index
ind = [1,2,3,6,7,8,9,10,13,14,15,16,17,20,21,22,23,24,27,28,29,30,31]

# Create map
map_morning_scooters = folium.Map(location = area_center, zoom_start = 12) 

# Add bus markers

for row_index, row_values in stops_in_promise.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    tool = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(location = loc, tooltip = tool, icon = icon) 

    marker.add_to(map_morning_scooters)

#Create for loop to add heat data to map

heat_data = [[[row['latitude'],row['longitude']] for index, row in may_geo_morning[may_geo_morning['pubdatetime'].apply(lambda x: x.day==i)].iterrows()] for i in ind]

hmwt = plugins.HeatMapWithTime(heat_data, index=ind, auto_play=True)

hmwt.add_to(map_morning_scooters)

# Add promise zone polygon

folium.GeoJson(promise_zone).add_to(map_morning_scooters)

# Save map

map_morning_scooters.save('map_morning_scooters.html')

# Show map

map_morning_scooters

The second map is showing the locations of the scooters in the evening over time. 

In [ ]:
# Create time index (defined above)

# Create map
map_evening_scooters = folium.Map(location = area_center, zoom_start = 12) 

# Add bus markers

for row_index, row_values in stops_in_promise.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    tool = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(location = loc, tooltip = tool, icon = icon) 

    marker.add_to(map_evening_scooters)

#Create for loop to add heat data to map

heat_data2 = [[[row['latitude'],row['longitude']] for index, row in may_geo_evening[may_geo_evening['pubdatetime'].apply(lambda x: x.day==i)].iterrows()] for i in ind]

hmwt2 = plugins.HeatMapWithTime(heat_data2, index=ind, auto_play=True)

hmwt2.add_to(map_evening_scooters)

# Add promise zone polygon

folium.GeoJson(promise_zone).add_to(map_evening_scooters)

# Save map



# Show map

map_evening_scooters